In [1]:
import pandas as pd
import seaborn as sns
import os
from googleapiclient.discovery import build

In [2]:
API_KEY = os.environ["YOUTUBE_API_KEY"]
youtube = build("youtube", "v3", developerKey=API_KEY)

In [3]:
def get_channel_id(service, username):
    response = service.channels().list(
        part="contentDetails,statistics",
        forUsername=username
    ).execute()
    return response["items"][0]["id"]

In [4]:
CHANNEL_ID = get_channel_id(youtube, "Felixje0")

In [5]:
def get_upload_id(service, channel_id):
    response = service.channels().list(
        part="contentDetails,statistics",
        id=channel_id
    ).execute()
    return response["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]

In [6]:
UPLOAD_ID = get_upload_id(youtube, CHANNEL_ID)

In [7]:
def get_video_ids(service, upload_id):
    video_ids = []

    response = service.playlistItems().list(
        part="contentDetails",
        playlistId=upload_id,
        maxResults=50
    ).execute()

    for video in response["items"]:
        video_ids.append(video["contentDetails"]["videoId"])
    
    next_page_token = response.get("nextPageToken")
    while next_page_token:
        response = service.playlistItems().list(
            part="contentDetails",
            playlistId=upload_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        for video in response["items"]:
            video_ids.append(video["contentDetails"]["videoId"])

        next_page_token = response.get("nextPageToken")

    return video_ids

In [8]:
video_ids = get_video_ids(youtube, UPLOAD_ID)
len(video_ids)

3819

In [9]:
def get_video_details(service, video_ids):
    result = []
    for i in range(0, len(video_ids), 50):
        response = service.videos().list(
            part="snippet,statistics",
            id=",".join(video_ids[i:i+50])
        ).execute()

        for video in response["items"]:
            try:
                result.append({
                    "title": video["snippet"]["title"],
                    "published_at": video["snippet"]["publishedAt"],
                    "description": video["snippet"]["description"],
                    "view_count": video["statistics"]["viewCount"],
                    "like_count": video["statistics"]["likeCount"],
                    "comments": video["statistics"]["commentCount"],
                })
            except Exception as _:
                result.append({
                    "title": video["snippet"]["title"],
                    "published_at": video["snippet"]["publishedAt"],
                    "description": video["snippet"]["description"],
                    "view_count": video["statistics"]["viewCount"],
                    "like_count": video["statistics"]["likeCount"],
                    "comments": 0,
                })
    df = pd.DataFrame(result)
    df["published_at"] = pd.to_datetime(df["published_at"])
    df["view_count"] = pd.to_numeric(df["view_count"])
    df["like_count"] = pd.to_numeric(df["like_count"])
    df["comments"] = pd.to_numeric(df["comments"])
    
    return df

In [10]:
video_stats = get_video_details(youtube, video_ids)
video_stats

,title,published_at,description,view_count,like_count,comments
0,StarCraft 2: EPIC Semi Finals - Maru vs herO! ...,2022-11-26 15:05:24+00:00,The best Terran facing off against the best Pr...,46478,2006,111
1,StarCraft 2: Why is he Playing BROOD WAR Units...,2022-11-25 15:00:04+00:00,Best-of-3 series of StarCraft 2 on the maps th...,43382,1273,89
2,StarCraft 2: MaxPax in Dark Templar PRISON! (E...,2022-11-24 16:00:34+00:00,Grand Finals of the ESL Open Cup 150 Europe in...,42827,1338,108
3,StarCraft 2: Dark is STYLING with Swarm Hosts!...,2022-11-23 15:00:15+00:00,Best-of-5 series of Zerg versus Protoss betwee...,60863,1634,120
4,"StarCraft 2: No Oracles, No Problems! (MaxPax ...",2022-11-22 15:00:14+00:00,Best-of-3 series of professional StarCraft 2 b...,41530,1491,85
...,...,...,...,...,...,...
3814,"★ Starcraft 2 FPVOD First Person Live Game: ""Z...",2011-09-30 13:28:15+00:00,More LowkoTV\n▶ Subscribe | http://www.lowkotv...,3139,30,12
3815,"Starcraft II Study #3 - ""My Heart For The Swar...",2011-09-27 16:45:18+00:00,More LowkoTV\n▶ Subscribe | http://www.lowkotv...,5031,41,12
3816,"★ Starcraft 2 Master: ""Multiplayer 1v1 Analysi...",2011-09-23 02:14:23+00:00,More LowkoTV\n▶ Subscribe | http://www.lowkotv...,4188,33,7
3817,"Starcraft II Study #2 - ""My Heart For The Swar...",2011-09-22 05:57:47+00:00,More LowkoTV\n▶ Subscribe | http://www.lowkotv...,6101,85,21


In [18]:
video_stats.to_csv("data/lowko_video_stats.csv", index=False)

In [22]:
video_stats = pd.read_csv("data/lowko_video_stats.csv").drop(columns=["Unnamed: 0"])
video_stats

,title,published_at,description,view_count,like_count,comments
0,StarCraft 2: EPIC Semi Finals - Maru vs herO! ...,2022-11-26 15:05:24+00:00,The best Terran facing off against the best Pr...,46478,2006,111
1,StarCraft 2: Why is he Playing BROOD WAR Units...,2022-11-25 15:00:04+00:00,Best-of-3 series of StarCraft 2 on the maps th...,43382,1273,89
2,StarCraft 2: MaxPax in Dark Templar PRISON! (E...,2022-11-24 16:00:34+00:00,Grand Finals of the ESL Open Cup 150 Europe in...,42827,1338,108
3,StarCraft 2: Dark is STYLING with Swarm Hosts!...,2022-11-23 15:00:15+00:00,Best-of-5 series of Zerg versus Protoss betwee...,60863,1634,120
4,"StarCraft 2: No Oracles, No Problems! (MaxPax ...",2022-11-22 15:00:14+00:00,Best-of-3 series of professional StarCraft 2 b...,41530,1491,85
...,...,...,...,...,...,...
3814,"★ Starcraft 2 FPVOD First Person Live Game: ""Z...",2011-09-30 13:28:15+00:00,More LowkoTV\n▶ Subscribe | http://www.lowkotv...,3139,30,12
3815,"Starcraft II Study #3 - ""My Heart For The Swar...",2011-09-27 16:45:18+00:00,More LowkoTV\n▶ Subscribe | http://www.lowkotv...,5031,41,12
3816,"★ Starcraft 2 Master: ""Multiplayer 1v1 Analysi...",2011-09-23 02:14:23+00:00,More LowkoTV\n▶ Subscribe | http://www.lowkotv...,4188,33,7
3817,"Starcraft II Study #2 - ""My Heart For The Swar...",2011-09-22 05:57:47+00:00,More LowkoTV\n▶ Subscribe | http://www.lowkotv...,6101,85,21
